In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
data=pd.read_csv("D:\\Workshops\\Machine Learning for Data Science & Artifcial Intelligence With Python\\data\\cardio.CSV")

In [ ]:
data.head()

In [ ]:
data.shape

# Missing Values & Duplicates

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data[["age","height","weight","ap_hi","ap_lo"]].boxplot()

# Some Feature Engineering

In [ ]:
data["age"]=(data["age"]/365).values.astype(int)

In [ ]:
data["bmi"]=data["weight"]/((data["height"]/100)**2)

In [ ]:
data.head()

In [ ]:
data.drop(["height","weight"],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
data[["age","ap_hi","ap_lo","bmi"]].boxplot()

# Outlier Handling

In [ ]:
data_num=data[["age","ap_hi","ap_lo","bmi"]]

In [ ]:
Q1 = data_num.quantile(0.25)
Q3 = data_num.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
out_rows=((data_num < (Q1 - 1.5 * IQR)) |(data_num > (Q3 + 1.5 * IQR))).any(axis=1) 
out_rows

In [ ]:
data=data[~out_rows]
data.head()

In [ ]:
data[["age","ap_hi","ap_lo","bmi"]].boxplot()

# Descriptive Analysis

In [ ]:
data[["age","ap_hi","ap_lo","bmi"]].describe()

In [ ]:
data["cardio"].value_counts()

In [ ]:
sns.countplot(data["cardio"])
plt.show()

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(x="age",hue="cardio",data=data)
plt.show()

In [ ]:
sns.boxplot(x="cardio",y="bmi",data=data)
plt.show()

In [ ]:
sns.heatmap(data[["age","ap_hi","ap_lo","bmi"]].corr(),annot=True,vmax=1,vmin=-1)
plt.show()

In [ ]:
sns.pairplot(data[["age","ap_hi","ap_lo","bmi"]])
plt.show()

In [ ]:
df=data[["age","ap_hi","ap_lo","bmi"]]
df["cardio"]=["Yes" if m==1 else "No" for m in data["cardio"]]

In [ ]:
df.head()

In [ ]:
sns.pairplot(df,hue="cardio")
plt.show()

# Machine Learning

In [ ]:
data.head()

In [ ]:
data.drop("id",axis=1,inplace=True)

In [ ]:
data=data.reindex(columns=["age","ap_hi","ap_lo","bmi","gender","cholesterol","gluc","smoke","alco","active","cardio"])

In [ ]:
data.head()

In [ ]:
data["gender"].value_counts()

In [ ]:
data["cholesterol"].value_counts()

In [ ]:
data["gluc"].value_counts()

In [ ]:
data["smoke"].value_counts()

In [ ]:
data["alco"].value_counts()

In [ ]:
data["active"].value_counts()

In [ ]:
data["cardio"].value_counts()

# Label Encoding

In [ ]:
le=LabelEncoder()
le

In [ ]:
data["gender"]=le.fit_transform(data["gender"])

In [ ]:
data.head()

# One Hot Encoding

In [ ]:
ohe=OneHotEncoder()
ohe

In [ ]:
ohot_encoded1=ohe.fit_transform(data["cholesterol"].values.reshape(len(data["cholesterol"].values),1)).toarray()
ohot_encoded1=ohot_encoded1[:,1:].astype(int)
df_ohot1=pd.DataFrame(ohot_encoded1,columns=["Above Nor Chol","Well Above Nor Chol"])

In [ ]:
df_ohot1.head()

In [ ]:
ohot_encoded2=ohe.fit_transform(data["gluc"].values.reshape(len(data["gluc"].values),1)).toarray()
ohot_encoded2=ohot_encoded2[:,1:].astype(int)
df_ohot2=pd.DataFrame(ohot_encoded2,columns=["Above Nor Gluc","Well Above Nor Gluc"])

In [ ]:
df_ohot2.head()

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.drop("index",axis=1,inplace=True)

In [ ]:
x=pd.concat([data.iloc[:,:10],df_ohot1,df_ohot2],axis=1)

In [ ]:
x.head()

In [ ]:
x.drop(["cholesterol","gluc"],axis=1,inplace=True)

In [ ]:
x=x.values

In [ ]:
y=data.iloc[:,10].values

# Standardizing Neumerical Features

In [ ]:
sc=StandardScaler()

In [ ]:
x[:,:4]=sc.fit_transform(x[:,:4])

# Train Test Splitting

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

# Modeling

In [ ]:
model1=KNeighborsClassifier(n_neighbors=50)
model2=LogisticRegression()
model3=SVC(kernel="rbf",C=1)
model4=RandomForestClassifier(n_estimators=500)

T1=("knn",model1)
T2=("lgr",model2)
T3=("svm",model3)
T4=("rf",model4)

model=VotingClassifier(estimators=[T1,T2,T3,T4],voting="hard")

In [ ]:
model.fit(x_train,y_train)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))